### 목표
- 타이타닉 데이터를 활용하여 생존자/사망자 예측
- kaggle 경진대회에 참가하여 점수를 확인
- 머신러닝의 전체 과정 체험

### 머신러닝 과정
- 1. 문제정의 : 목표설정, 어떤 모델
  2. 데이터 수집 : 분류를 할거라면 class 로 담긴 레이블이 있는 데이터 수집
  3. 데이터 전처리 : 이상치/결측치 처리, 특성 처리(특성공학)
  4. 데이터 탐색(탐색적 데이터 분석) : 기술 통계 확인, 특성 간의 관계 파악
  5. 모델 선택 및 하이퍼 파라미터 선정
  6. 모델 예측 및 평가
  7. 모델 서비스

In [ ]:
#train- 학습용 문제, 답
#test - 테스트용 문제  -> 답은 kaggle 가지고 있음
# pre 잘했는지 평가하기 위해서 kaggle 업로드
#submission - pre를 제출하기 위한 답안지 파일 

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # 시각화 라이브러리
from sklearn.model_selection import train_test_split # train, test 분리하는 도구
from sklearn.metrics import accuracy_score # 평가도구
#tree model - 사망/생존 이진분류
from sklearn.tree import DecisionTreeClassifier

In [55]:
# train.test 변수에 데이터 불러와서 담기
# 인덱스를 승객의 번호로 설정해서 담아 줄 것

test = pd.read_csv('data/titanic/test.csv', index_col = 'PassengerId')
train = pd.read_csv('data/titanic/train.csv', index_col = 'PassengerId')

In [57]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [58]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


 0   Survived  - 생존/ 사망 여부 (답 데이터, Class Label)  
 1   Pclass    - 승객의 등급(1- 귀족층,2- 중간층,3- 하위계층)  
 2   Name      - 이름        
 3   Sex       - 성별(male, female)
 4   Age       - 나이(0~  
 5   SibSp     - 형제자매+배우자 수  
 6   Parch     - 부모 + 자식 수  
 7   Ticket    - 티겟 번호  
 8   Fare      - 요금  
 9   Cabin     - 객실번호  
 10  Embarked  - 승선항  

In [59]:
# 크기확인
print('트레인 크기 :', train.shape)
print('테스트 크기 :', test.shape) # test 에는 정답 데이터가 없기 때문에 열의 갯수가 차이가 난다.

트레인 크기 : (891, 11)
테스트 크기 : (418, 10)


In [60]:
# 컬럼명만 뽑아서 확인하기
print(train.columns)
print(test.columns)

# 컬럼의 순서도 중요하다.
# 컬럼의 이름도 동일해야 한다.

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


### 데이터 전처리 및 데이터 탐색
- 결측치 확인


In [61]:
train.info()
print('트레인 결측치 컬럼 : Embarked, Age, Cabin')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
트레인 결측치 컬럼 : Embarked, Age, Cabin


In [62]:
# 판다스 함수를 통해서 test 결측치 확인
print(test.isnull().sum())
print('결측치 존재 컬럼 : Fare, Age, Cabin')

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64
결측치 존재 컬럼 : Fare, Age, Cabin


### train-Embarked 결측치 처리
- 승객이 탑승한 항구 이름

In [63]:
# train 데이터 기준으로 Embarked 컬럼 확인
# nan 결측치
train['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [64]:
# 빈도수가 높은(최빈값) 승선항 알파벳으로 결측치를 처리
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [65]:
# Embarked 결측치를 대문자 S로 채운다.
# fillna(채울값)
train['Embarked'] = train['Embarked'].fillna('S')
train['Embarked'].unique() # non 결측치가 제거 된 것을 확인 할 수 있다.
train['Embarked'].value_counts() # S 갯수가 644 -> 646 으로 변경 되었다.

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
# 탐색용 데이터는 train 이다.(test가 아니다)
# 결측치를 채워주는 기준은 train 이다
# 모델 학습 : 모델이 학습을 잘하도록 데이터를 셋팅
# test 데이터는 모델을 일반화시키는데 도움이 되지 않는다.
# train 데이터가 학습할 때 사용되고, 모델의 일반화에 도움을 준다.

In [ ]:
# 추가) Embarked 컬럼에 결측치가 있던 행의 정보를 확인하고,
# 비슷한 사람의 데이터가 있다면 그 사람의 승선상 정보를 대체값으로 활용

### test-Fare 결측치 처리
- 요금 평균, 중앙값 계산 가능
- 데이터 분석을 진행하면서 관련이 있을 법한 컬럼을 연관지어서 대체값을 확인해보자

In [66]:
# train 기준으로 데이터를 탐색
# Fare 컬럼 정보 확인 -> 기술통계량
train['Fare'].describe()

# 요금의 대표값으로 평균보다 중앙이 더 나을 수 있겠다

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [ ]:
# 변수(특성) 간의 상관관계 확인
# 상관계수(숫자) : -1 ~ 1
# 각 컬럼의 상관 정도를 수치로 표현한 값 : 상관계수
# -1, 1 에 가까울수록 변수간의 관계성이 있다.
# 0에 가까울수록 변수간의 관계성이 적다.

In [67]:
# 생존 사망과 연관이 있는 컬럼 확인 해보기
# corr() : 상관계수 반환 메서드
# abs() : 절대값으로 변환 메서

train.corr()['Survived'].abs()
# Pclass 가 생존/사망에 연관성이 있을 수 있겠다라는 추측이 가능하다.
# 등급에 따라 요금의 중앙값으로 채워주는 것도 의미가 있지 않을까?

C:\Users\gjaischool1\AppData\Local\Temp\ipykernel_5664\2465632766.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()['Survived'].abs()


Survived    1.000000
Pclass      0.338481
Age         0.077221
SibSp       0.035322
Parch       0.081629
Fare        0.257307
Name: Survived, dtype: float64

In [79]:
# Pclass, Fare 두개 컬럼만 인덱싱 해보기
# Pclass 1,2,3 등급끼리 묶기(그룹화하기)
# median() 집계함수 연결
# 집계함수 : count(), mean(), min(), max(), median() sum()
train[['Pclass','Fare']].groupby('Pclass').median()

,Fare
Pclass,
1,60.2875
2,14.2500
3,8.0500
